
# Implementation of Simple Vegetation-Morphodynamic Interactions

<img src="DALL·E 2024-08-02 00.04.21 - A fantastical and over-the-top scene showcasing the awesomeness of coastal vegetation. The scene includes lush, vibrant coastal plants, with lasers sh.webp" style="width: 500px;" /> | 
<img src="DALL·E 2024-08-02 17.00.35 - A fantastical scene showcasing the awesomeness of coastal vegetation, with a more realistic depiction of the plants. The scene includes lush, vibrant .webp" style="width: 500px;" /> |
<img src="DALL·E 2024-08-02 17.01.47 - A fantastical and over-the-top scene showcasing the awesomeness of coastal vegetation with a focus on realism. The scene includes lush, realistic marr.webp" style="width: 500px;" /> |
<img src="DALL·E 2024-08-02 17.11.49 - A fantastical and over-the-top scene showcasing the awesomeness of coastal vegetation, with more realistic elements. The scene includes lush, vibrant .webp" style="width: 500px;" /> |

The main objective of this notebook is to provide insight into how vegetation can help in building dunes and providing added value to people living along the coast. Throughout this notebook, we'll explore how to implement the influence and growth of vegetation in a simple manner. With the produced results, we hope to answer the following questions:

* How does vegetation in coastal environments impact the physical geomorphological development of the system?
* How can active vegetation management help in shaping the coast and providing additional benefits?

## Case study: Building a dune!

Our case study focuses on a house located along a beach. Currently, the only barrier between the house and the ocean is the beach itself. Consequently, the homeowner faces nuisances from blown sand and a lack of protection against flooding during extreme hydrodynamic events.

To address these issues, the homeowner wishes to establish a dune or sand buffer in front of the house, without obstructing the sea view. Several design alternatives are available for creating such a dune. The evaluation will be based on three scenarios:

1. **Benchmark:** No sand-capturing implementation.
2. **Sand Fences:** A cheap but static solution.
3. **Vegetation:** A dynamic approach.

<img src="case-study.png" />

## Method: Coupling custom Python functions with an existing aeolian model

We are going to use a simple semi-2D AeoLiS model of the coastal profile to describe the onshore aeolian flux and investigate how different solutions could affect these fluxes. The AeoLiS model is used as a basis for the computation of the supply-limited onshore transport from the intertidal area and beach towards the dune and house. 

The implementation of designs is done using linking custom Python functions to AeoLiS using BMI. This way, we can 1) showcase the relative biophysical interactions between vegetation and morphodynamics, and 2) demonstrate how to implement relatively simple functions to a numerical model without having to touch the source code.

<img src="vegetation-coupling-method.png" />






# Vegetation functions

The AeoLiS model handles the supply-limited aeolian transport from the intertidal zone to the beach and dunes. While AeoLiS already includes functionalities to describe the influence of vegetation, we aim to demonstrate this impact on geomorphology in the simplest way possible. Additionally, we will showcase how to use the Basic Model Interface (BMI) to integrate custom Python functions with an existing BMI-compatible model.

To achieve this, we will create custom functions to describe the following processes:

1. **Initialization**: The initial establishment of vegetation.
2. **Development**: The growth and burial of vegetation over time.
3. **Shear reduction**: The reduction of shear due to the presence of vegetation.


### Initialization

The presence of vegetation (and sand fences) is quantitatively described by height ($h$), determining the extent to which the vegetation influences the spatial shear stress. We assume that the height ($h$) has a maximum value of 1 m.

During the initialization of either vegetation or sand fences, we will populate individual cells with starting values for height and the corresponding density:

In [34]:
def vegetation_init(i_vegetated, height_init):
    """
    Initialize vegetation height.

    Parameters:
    i_vegetated (array-like): Boolean array indicatnig vegetated cells.
    height_init (float): Initial height of the vegetation (or sand fences).

    Returns:
    tuple: Array representing vegetation height.
    """

    # Initialize vegetation height array with zeros
    vegetation_height = np.zeros(np.shape(i_vegetated))

    # Set initial height for vegetated areas
    vegetation_height[i_vegetated] = height_init

    return vegetation_height


### Development: Growth and Burial

The `vegetation_development` function updates the vegetation height based on growth rate, timestep, and changes in bed level. Dune vegetation is known for its ability to grow with sedimentation. Although marram grass is more effective in accreting conditions compared to erosive ones, this function simplifies the process by assuming optimal growth (equal to the growth rate) in static conditions (i.e., when the bed level change is zero).

The growth rate serves as a maximum potential growth rate and is provided in meters per year. It is converted to meters per second to match the timestep, which is given in seconds. This function can also be applied to sand fences, but with a growth rate of zero, since these are static structures.

The function calculates the vegetation height at each timestep by considering both vegetation growth and changes in bed level (e.g., sediment deposition or erosion). The result is the updated vegetation height, accounting for both growth and burial processes.

In [106]:
def vegetation_development(growth_rate, timestep, vegetation_height, i_vegetated, bed_level_change):
    """
    Update vegetation height based on growth rate, timestep, and bed level changes.

    Parameters:
    growth_rate (float): Annual growth rate of the vegetation (m/year).
    timestep (float): Length of the timestep (seconds).
    vegetation_height (array-like): Current heights of the vegetation.
    bed_level_change (array-like): Changes in bed level during the timestep.

    Returns:
    array-like: Updated heights of the vegetation.
    """

    # Convert the growth rate from m/year to m/timestep
    growth_rate_per_second = growth_rate / (365 * 24 * 3600)
    growth_rate_timestep = growth_rate_per_second * timestep

    # Update vegetation height considering growth and bed level changes
    vegetation_height[i_vegetated] += growth_rate_timestep - np.abs(bed_level_change[i_vegetated])

    # Ensure that vegetation height does not exceed the maximum of 1 meter and is not lower than 0
    vegetation_height = np.clip(vegetation_height, 0, 1)

    return vegetation_height
    

### Shear reduction

Vegetation (and sand fences) impact the geomorphological development of coasts by reducing local shear stresses. This results in a gradient in transport and therefore affects erosion and sedimentation. Due to the construction of the AeoLiS model, it is easier to increase the velocity threshold ($u_{th}$) rather than reducing shear stress ($u_*$). Using the principles from Bagnold (1937), the impact on the aeolian transport ($Q$) will be approximately the same:

$Q = f( (u_* - u_{th})^3$ )

The magnitude of increase is estimated using the approach by Raupach. Here, Γ is a scaling parameter, with a default value of 16. The result will be stored as a mask that will be multiplied with the current velocity threshold in AeoLiS:

$mask_{uth} =  1 + Γ * h_{veg}$

Within AeoLiS, the velocity threshold is adjusted as follows:

$uth =  uth * mask_{uth}$



In [23]:
def adjust_velocity_threshold(vegetation_height, gamma=16.0):
    """
    Adjusts the velocity threshold based on vegetation height.

    Parameters:
    vegetation_height (array-like): Heights of the vegetation.
    gamma (float, optional): Scaling parameter, default is 16.0.

    Returns:
    array-like: Adjusted velocity threshold mask.
    """

    # Compute the velocity threshold mask
    velocity_threshold_mask = 1 + gamma * vegetation_height

    return velocity_threshold_mask
    

# Let's start modelling!

First, import the required packages:

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
from ipywidgets import IntProgress
from IPython.display import display
import time
from aeolis.model import AeoLiSRunner
from IPython.display import HTML

Now, set the timing settings that determines for how long and with which timestep we will run AeoLiS:

In [82]:
# Timing settigns
timestep = 3600.       # seconds (= 1 hour), timestep to run the model and exchange vegetation information
start_time = 0.        # seconds (= 0 year), start time of the model
end_time = 31536000.   # seconds (= 1 year), end time of the model
output_time = 604800.  # seconds (= 1 week), the frequency of plotting output (and animation)

Setting the parameters for our vegetation functions:

In [99]:
height_init = 1.0             # Initial height of vegetation (or sand fence) in meters
growth_rate = 0.0             # Growth rate of vegetation in meters per year
gamma = 16.0                  # Scaling parameter for adjusting velocity threshold
x_min_vegetation = 400.0      # Cross-shore starting location for vegetation presence relative to the offshore boundary (in meters)
x_max_vegetation = 450.0      # Cross-shore ending location for vegetation presence relative to the offshore boundary (in meters)

cell_width = 5  # m 
cell_area = cell_width**2.

Determine the location of the AeoLiS configuration file and change the working directory:

In [29]:
# Find file in the same directory as this script that contains aeolis.txt
configfile = '../setup/aeolis.txt'
os.chdir(os.path.dirname(configfile))

Create the BMI-AeoLiS wrapper and initialize the model:

Now let's start updating the model and call the vegetation functions. 

In [111]:
# Create AeoLiS BMI Wrapper
aeolis_wrapper = AeoLiSRunner(configfile)

# Initialize the wrapper
aeolis_wrapper.initialize()

# Get variables from the initial AeoLiS state
x = aeolis_wrapper.get_var('x')
y = aeolis_wrapper.get_var('y')
bed_level = aeolis_wrapper.get_var('zb')
domain_width = np.shape(x)[0] * cell_width

# Initialize vegetation
i_vegetated = (x > x_min_vegetation) * (x < x_max_vegetation)
vegetation_height = vegetation_init(i_vegetated, height_init)

# Initialize the progress bar
f = IntProgress(min=start_time, max=end_time) # instantiate the bar
display(f) # display the bar

# Loop over all timesteps to run the model
times = np.arange(start_time, end_time, timestep)
output_times = np.arange(start_time, end_time, output_time)

for t in times:

    # Update velocity threshold
    velocity_threshold_mask = adjust_velocity_threshold(vegetation_height, gamma=16.0)

    # Set updated velocity threshold to AeoLiS
    aeolis_wrapper.set_var('threshold_mask', velocity_threshold_mask)

    # Update AeoLiS
    aeolis_wrapper.update(timestep)

    # Get updated variables from AeoLiS
    bed_level = aeolis_wrapper.get_var('zb')          # Get the updated bed level
    transport = aeolis_wrapper.get_var('q')           # Get the sediment transport
    bed_level_change = aeolis_wrapper.get_var('dzb')  # Get the bed level change

    # Update vegetation development
    vegetation_height = vegetation_development(growth_rate, timestep, vegetation_height, i_vegetated, bed_level_change)

    # --------- Start of saving output --------
    
    # Compute the amount of transport that is being transported across the onshore boundary and dune growth
    volume_change = np.sum(bed_level_change) * cell_area / domain_width  # Growth in m^3/m/timestep
    onshore_bc_flux_volume = timestep * (np.sum(transport[:, -2, 0]) / cell_width) / (2650 * (1 - 0.4)) # Convert from kg/m/s to m3/m 
    
    # Store variables in timestacks for animation purposes
    if t == start_time:  # Initialize timestacks on the first iteration
        bed_level_times = [bed_level.copy()]
        vegetation_height_times = [vegetation_height.copy()]
        transport_times = [transport.copy()]
        onshore_bc_flux_times = [onshore_bc_flux_volume]
        volume_change_times = [volume_change]
        plot_times = [t]
    
    elif t % output_time == 0:  # Append to timestacks at specified intervals
        bed_level_times.append(bed_level.copy())
        vegetation_height_times.append(vegetation_height.copy())
        transport_times.append(transport.copy())
        
    onshore_bc_flux_times.append(onshore_bc_flux_times[-1] + onshore_bc_flux_volume)
    volume_change_times.append(volume_change_times[-1] + volume_change)
    plot_times.append(t)

    # Signal to increment the progress bar
    f.value += timestep 
    
# Finalize the wrapper
aeolis_wrapper.finalize()

IntProgress(value=0, max=31536000)

# Plotting the results

Now the model is finished, we will animate the results by plotting the variables stored in the timestacks:



In [112]:
def animate(it):
    """
    Update the plot for the given frame index `it`.

    Parameters:
    it (int): Frame index for the animation.

    Returns:
    None
    """
    
    line_zb.set_ydata(zb_cross[it, :])
    line_hv.set_ydata(hv_cross[it, :] + zb_cross[it, :])
    line_q.set_ydata(qs_cross[it, :])

    # Signal to increment the progress bar
    f.value += 1 
    
    return fill_zb, fill_hv, line_hw, line_q

# Initialize the progress bar
f = IntProgress(min=0, max=len(output_times)) # instantiate the bar
display(f) # display the bar

# Get cross-shore transects from 2D domains by averaging
x_cross = np.average(x, axis=0)
y_cross = np.average(y, axis=0)
zb_cross = np.average(bed_level_times, axis=1)
hv_cross = np.average(vegetation_height_times, axis=1)
qs_cross = np.average(transport_times, axis=1)

# Initialize the figure for plotting
fig, axs = plt.subplots(2, 1, figsize=(10, 6))
plt.close()

# Plot the cross-shore profile: Initial bed level, bed level, vegetation height (and water level)
ax0 = axs[0]
ax0.set_title('Cross-shore Profile')
line_hv, = ax0.plot(x_cross, hv_cross[0, :] + zb_cross[0, :], color='green')
line_zb, = ax0.plot(x_cross, zb_cross[0, :], color='orange')
ax0.plot(x_cross, zb_cross[0, :], 'k:', alpha=0.2)
ax0.axhline(0., color='b', alpha=0.2)
ax0.set_xlabel('Cross-shore distance, x [m]')
ax0.set_ylabel('Elevation, z [m+MSL]')
ax0.set_xlim([0, 500])
ax0.set_ylim([-2, 16])

# # Plot the cross-shore distribution of onshore transport
# ax1 = axs[1]
# ax1.set_title('Cross-shore Distribution of Onshore Transport')
# line_q, = ax1.plot(x_cross, qs_cross[0, :], 'k', alpha=0.2)
# ax1.set_xlabel('Cross-shore distance, x [m]')
# ax1.set_ylabel('Transport rate, q [m^3/m/s]')

# Plot the cumulative onshore transport across the onshore boundary
ax2 = axs[1]
ax2.set_title('Cumulative Onshore Transport and Dune Growth')
ax2.plot(plot_times, onshore_bc_flux_times, color='b', alpha=0.5, label='Outgoing onshore flux')
ax2.plot(plot_times, volume_change_times, color='r', alpha=0.5, label='Dune growth')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Sand volume [m^3/m]')
ax2.legend()

# Create animation
ani = animation.FuncAnimation(fig, animate, repeat=False, frames=len(output_times))

HTML(ani.to_jshtml())

IntProgress(value=0, max=53)

# Model interpretation / Case 0: Benchmark scenario

Questions: Model setup interpretation. Wind speed? Direction? Profile shape? Tidal excursion? Size? Slope?

In the current scenario (initial settings of this Notebook), no features are actively capturing sediment on the beach. As a result, all the sediment is blown directly exiting the domain and blown against the house. Run the model and try to answer the questions below based on the animated results:

- How much sand is transported across the onshore boundary?
- Where is this sediment picked up from?

# Case 1: Sand Fences

Now we are adding sand fences... xmin=... xmax=... Growth rate is zero, initial height is one. Modify these parameters!

- Describe what happens? How long effective? ...

# Case 2: Vegetation

Now vegetation: vegetation growth rate = 2 m/year (?)

# Case 2b: Different vegetation type

Different type to create different dune shape (=0.5 m/year) but wider coverage. Influence on dune shape?